In [14]:
!pip3 install keras

  Using cached https://files.pythonhosted.org/packages/5e/10/aa32dad071ce52b5502266b5c659451cfd6ffcbf14e6c8c4f16c0ff5aaab/Keras-2.2.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/90/85/64c82949765cfb246bbdaf5aca2d55f400f792655927a017710a78445def/Keras_Applications-1.0.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c0/bf/0315ef6a9fd3fc2346e85b0ff1f5f83ca17073f2c31ac719ab2e4da0d4a3/Keras_Preprocessing-1.0.9-py2.py3-none-any.whl


---

This Notebook shows implementation of SSN Stacker.

In [1]:
BUCKET_NAME = "msil_raw"
FOLDER_NAME = "training_data"
XGBSTACKER = "stack_xgb_data.csv"
LSTMSTACKER = "stack_lstm_data.csv"

In [2]:
import google.datalab.storage as storage
import pandas as pd
from io import BytesIO

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import time
from datetime import datetime
from scipy import integrate
import pickle
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers

/usr/local/envs/py3env/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# setting up the parameters
plt.rcParams["figure.figsize"] = (10, 10)
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)
pd.set_option("precision", 15)
sns.set_style("darkgrid")

In [4]:
mybucket = storage.Bucket(BUCKET_NAME)
data_csv = mybucket.object(FOLDER_NAME + "/" + XGBSTACKER)

uri = data_csv.uri
%gcs read --object $uri --variable data

stack_01 = pd.read_csv(BytesIO(data))
stack_01.head()

,y,yhat
0,93.0,92.999488800764084
1,93.0,92.998987704515457
2,93.0,92.998491972684860
3,93.0,92.997980743646622
4,93.0,92.997486114501953


In [5]:
mybucket = storage.Bucket(BUCKET_NAME)
data_csv = mybucket.object(FOLDER_NAME + "/" + LSTMSTACKER)

uri = data_csv.uri
%gcs read --object $uri --variable data

stack_02 = pd.read_csv(BytesIO(data))
stack_02.head()

,y,yhat
0,93.0,92.997296408284456
1,93.0,92.994592540198937
2,93.0,92.991889250697554
3,93.0,92.989186197984964
4,93.0,92.986482355510788


In [6]:
print("length of XGB Predictions is  {}".format(len(stack_01)))
print("length of LSTM Predictions is  {}".format(len(stack_02)))

length of XGB Predictions is  2203313
length of LSTM Predictions is  3032534


In [7]:
# making both dataset of same len
stack_02 = stack_02[:2203313]

In [8]:
stack_02.columns = ["actual2", "LSTM_pred"]
stack_01.columns = ["actual", "XGB_pred"]

df = pd.concat((stack_01, stack_02), axis = 1).reset_index(drop = True)
df = df.drop(columns = ["actual2"])

df = df[["XGB_pred", "LSTM_pred", "actual"]]
df.columns = ["XGB_pred", "LSTM_pred", "label"]
df.sample(10)

,XGB_pred,LSTM_pred,label
1346797,54.042679941654214,62.650685081863784,54.466497418140818
572358,51.111947709321967,53.181052082311361,51.027956732713278
1866146,35.333653455972659,51.040432356670493,33.492111107775180
1171115,83.556290346384046,83.632466423278672,82.343033023102677
1272661,83.300452572107318,84.680283159762624,82.998774052755095
2130054,89.399169260263463,89.393875659769407,89.399999999999977
1723166,63.656396609544757,65.485749542294073,60.313712030667617
1045753,79.769128417968730,74.544420389831060,79.870736540089752
2057520,61.463809067010892,66.063826253963640,59.460799055816771
1613779,85.640096795558932,85.282950522657487,86.019164949813955


In [9]:
df.isnull().sum()

XGB_pred     0
LSTM_pred    0
label        0
dtype: int64

In [ ]:
x = df[["XGB_pred", "LSTM_pred"]].values
y = df["label"].values.reshape(-1, 1)

mms_1 = MinMaxScaler(feature_range = (0, 1))
mms_2 = MinMaxScaler(feature_range = (0, 1))
mms_1 = mms_1.fit(x)
mms_2 = mms_2.fit(y)
pickle.dump(mms_1, open("ssn_scaler_x.pickle.dat", "wb"))
pickle.dump(mms_2, open("ssn_scaler_y.pickle.dat", "wb"))

x=  mms_2.transform(x)
y = mms_2.transform(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)

In [11]:
model = Sequential()
model.add(Dense(128, input_dim = x_train.shape[1], kernel_initializer = 'normal', activation = 'relu'))
model.add(Dense(16, kernel_initializer = 'normal', activation = 'relu'))
model.add(Dense(1, kernel_initializer = 'normal'))
# Compile model
model.compile(loss='mean_squared_error', optimizer="adam")

In [12]:
model.fit(x_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
1652484/1652484 [==============================] - 100s 61us/step - loss: 0.0011
Epoch 2/10
1652484/1652484 [==============================] - 103s 62us/step - loss: 6.4276e-04
Epoch 3/10
1652484/1652484 [==============================] - 102s 62us/step - loss: 6.1744e-04
Epoch 4/10
1652484/1652484 [==============================] - 99s 60us/step - loss: 6.0474e-04
Epoch 5/10
1652484/1652484 [==============================] - 100s 60us/step - loss: 5.9337e-04
Epoch 6/10
1652484/1652484 [==============================] - 100s 60us/step - loss: 5.5266e-04
Epoch 7/10
1652484/1652484 [==============================] - 101s 61us/step - loss: 5.5421e-04
Epoch 8/10
1652484/1652484 [==============================] - 93s 56us/step - loss: 5.2650e-04
Epoch 9/10
1652484/1652484 [==============================] - 101s 61us/step - loss: 5.2130e-04
Epoch 10/10
1652484/1652484 [==============================] - 98s 59us/step - loss: 4.9639e-04


In [13]:
predictions = model.predict(x_test)
predictions = mms_2.inverse_transform(predictions)
y_test = mms_2.inverse_transform(y_test)

In [14]:
predictions = predictions.ravel()
y_test = y_test.ravel()

In [16]:
result_df = pd.DataFrame({
  "y": y_test,
  "yhat": predictions
})

In [17]:
result_df.sample(10)

,y,yhat
462879,83.917281984139620,82.828643798828125
454178,72.429146470143010,70.737236022949219
550137,64.799999999999997,65.131294250488281
22096,89.872184440870271,89.372619628906250
423203,61.197558209820180,64.281372070312500
7391,34.547774969806085,35.281394958496094
480516,90.474506471885420,90.730033874511719
413620,80.566038708233791,81.444915771484375
343872,72.570940811048615,72.817901611328125
323251,44.071349805958732,43.463050842285156


In [18]:
(((result_df["y"] - result_df["yhat"])**2).mean())**0.5

2.223739796964447

## Saving the Model

In [19]:
# serialize model to JSON
model_json = model.to_json()
with open("ann_test.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("ann_test.h5")
print("Model is saved !!!")

Model is saved !!!


In [42]:
!gsutil cp 'model_lstm_stack.json' 'gs://msil_raw/training_data/model_lstm_stack_final.json'

Copying file://model_lstm_stack.json [Content-Type=application/json]...
/ [1 files][  1.5 KiB/  1.5 KiB]                                                
Operation completed over 1 objects/1.5 KiB.                                      
